In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


In [5]:
dados = pd.read_csv('ibov.csv')
dados

,DATA,ABERTURA,FECHAMENTO,VARIAÇÃO,MÍNIMO,MÁXIMO
0,19/08/2025,137.321,134.432,"-2,10",133.997,137.321
1,18/08/2025,n/d,137.322,"0,72",136.341,137.902
2,15/08/2025,136.354,136.341,"-0,25",135.583,136.431
3,13/08/2025,135.629,136.687,"-0,89",136.535,137.913
4,12/08/2025,135.913,137.914,"1,69",135.629,138.414
5,11/08/2025,135.913,135.623,"-0,21",135.496,136.307
6,08/08/2025,136.526,135.913,"-0,45",135.659,136.761
7,07/08/2025,133.169,136.528,"1,48",134.533,137.007
8,06/08/2025,133.169,134.538,"1,04",133.169,135.241
9,05/08/2025,132.440,133.151,"0,14",132.682,134.233


In [6]:
numeric_cols = dados.select_dtypes(include=[np.number]).columns
if  len(numeric_cols) == 0:
  raise ValueError("⚠ Nenhuma coluna numérica encontrada no CSV.")
forecast_col = numeric_cols[0]
print(f"📊 Coluna escolhida para previsão: {forecast_col}")


📊 Coluna escolhida para previsão: FECHAMENTO


In [8]:
def prepare_data(dados, forecast_col, forecast_out, test_size):
    # Criando a coluna alvo (y), deslocando as últimas linhas para prever
    y = dados[forecast_col].shift(-forecast_out)

    # Criando a matriz de recursos (X)
    X = np.array(dados[[forecast_col]])
    X = preprocessing.scale(X)

    # Separando a parte para previsão futura
    X_lately = X[-forecast_out:]

    # Removendo as últimas linhas para manter consistência
    X = X[:-forecast_out]
    y = y[:-forecast_out]

    # Separando treino e teste
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, y, test_size=test_size, random_state=0
    )

    return X_train, X_test, Y_train, Y_test, X_lately

# Configurações
forecast_out = 5   # número de passos à frente
test_size = 0.2    # 20% para teste

# Preparando os dados
X_train, X_test, Y_train, Y_test, X_lately = prepare_data(dados, forecast_col, forecast_out, test_size)

# Criando e treinando o modelo
model = LinearRegression()
model.fit(X_train, Y_train)

# Avaliando o modelo
accuracy = model.score(X_test, Y_test)
print(f"Acurácia do modelo: {accuracy:.2f}")

# Fazendo previsões futuras
forecast = model.predict(X_lately)
print("🔮 Previsões futuras:", forecast)


Acurácia do modelo: 0.05
🔮 Previsões futuras: [136.36568162 136.59349852 136.3326928  136.06412501 135.77809255]
